# Scrapy

<img src ="files/images/Scrapy_logo.jpg">

Scrapy  является основой приложения для обхода веб-сайтов и извлечения структурированных данных, которые могут быть использованы для широкого спектра полезных приложений, таких как интеллектуальный анализ данных, обработки информации или архивирования истории.


Scrapy – одна из наиболее популярных и производительных библиотек Python для получения данных с веб-страниц, которая включает в себя большинство общих функциональных возможностей. Это значит, что вам не придётся самостоятельно прописывать многие функции. Scrapy позволяет быстро и без труда создать «веб-паука».

Официальная документация: https://docs.scrapy.org/en/latest/index.html

## Установка

С помощью pip:

In [ ]:
pip install Scrapy

<img src ="files/images/cmd2.png">

Однако, установить библиотеку через pip никик не получалось, поэтому проишлось устанавливать Anacondas и работать через неё.

In [ ]:
conda install -c conda-forge scrapy

## Создание проекта

Официальная документация: https://docs.scrapy.org/en/latest/intro/tutorial.html

До того, как начать scraping, вам необходимо создать новый Scrapy прект (Scrapy project). Перейдите в директорию, где вы хотите разместить ваш проект и выполните:

In [ ]:
scrapy startproject tutorial

Получится:

<img src ="files/images/cmd3.png">

<img src ="files/images/1.png">

<img src ="files/images/2.png">

Например middlewares.py:

<img src ="files/images/3.png">

## Создание паука

Пауки - это классы, которые определяет пользователь, и которые Scrapy использует для сбора данных (scraping) с веб-сайта (или группы веб-сайтов). Они должны ,быть подклассом  scrapy.Spider и определять первоначальные запросы и методы, чтобы переходить по ссылкам на страницах и  анализировать  содержимое страницы для извлечения данных.

Создадим паука first_spider в папке /spiders:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "first_spider"

    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/page/1/',
            'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log('Saved file %s' % filename)

Как можно заметить сощданный паук ябляется подклассом scrapy.Spider и определяет некоторые атрибуты и методы:
- name: определяет паука. Оно должно быть уникальным для каждого паука.
- start_requests(): метод должен возвращать итеририруемые запросы (можно возвращать список запросов), по которые паук и начнёт проходить. Последующие запросы будут создаваться последовательно на основе этих первоначальных запросов.
- parse(): метод, который будет вызван для обработки ответа, загруженного для каждого из сделанных запросов. Параметр response является экземпляром TextResponse, который содержит содержимое страницы и имеет дополнительные полезные методы для его обработки. parse() - метод, который просматривает ответ, поочерёдно извлекая необходимые данные, а также находит новые URL-адреса для продолжния и создаёт новые запросы для них.

В данном примере паук first_spider собирает данные с http://quotes.toscrape.com

Данный код можно немного сократить:

In [ ]:
class QuotesSpider(scrapy.Spider):
    name = "first_spider"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)

Теперь мы отдельно не определяем мотод start_requests(), а передаём необходимые URL-адреса как метод. 

### Запуск паука

In [ ]:
scrapy crawl first_spider

<img src ="files/images/cmd4.png">

В папке spiders можно увидеть два новых файла quotes1-html и quotes2-html, содержащие соответствующие URL-адреса, как было указано в методе parse()
<img src ="files/images/4.png">

Scrapy собирает объекты типа scrapy.Request, возвращённые методом start_requests(). Получив ответ для каждого из них, он создает экземпляры объектов ответа и вызывает метод обратного вызова, связанный с запросом (в данном случае, метод parse), передавая ответ в качестве аргумента.

## Извлечение данных

Лучший способ научиться извлекать данные с помощью Scrapy - это использование селекторов с помощью оболлочки  Scrapy shell.

In [ ]:
scrapy shell "http://quotes.toscrape.com/page/1/"

<img src ="files/images/cmd5.png">

Запустим response.css("title")
<img src ="files/images/cmd6.png">

"Selector xpath='descendant-or-self::title' data='<title>Quotes to Scrape</title>'" - объект SelectorList, который показывает пользователю список селекторов, что позволяет далее исследовать XML/HTML элементы, выполнять более мелкие селекторы и извлекать данные.

In [ ]:
response.css('title::text').extract()
['Quotes to Scrape']

In [ ]:
response.css('title').extract()
[<title>Quotes to Scrape</title>]

In [ ]:
response.css('title::text').extract_first()
'Quotes to Scrape'

In [ ]:
response.css('title::text').extract_first()
'Quotes to Scrape'

<img src ="files/images/cmd7.png">

### Извлечение цитат и аторов

Если мы запустим 
response.css("div.quote"), Scrapy выведет список из десяти объектов исследуемой страницы.
<img src ="files/images/cmd8.png">

Исследуем первый из них. 
<img src ="files/images/5.png">
Выполним следущие команды:
1. quote = response.css("div.quote")[0]
2. title = quote.css("span.text::text").extract_first()
3. author = quote.css("small.author::text").extract_first()
<img src ="files/images/cmd9.png">

## Перепишем паука:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "first_spider"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('small.author::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

<img src ="files/images/cmd10.png">

2018-11-11 20:21:26 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'tags': ['abilities', 'choices']}

## Запишем полученные данае в файл quotes.json

Запустим:

In [ ]:
scrapy crawl first_spider -o quotes.json

<img src ="files/images/6.png">
<img src ="files/images/7.png">

При необходимости данные можно записывать в файлы с расширениями 'json', 'jsonlines', 'jl', 'csv', 'xml', 'marshal', 'pickle'

## Чтение нескольких страниц

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "first_spider"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('small.author::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

        next_page = response.css('li.next a::attr(href)').extract_first()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

<img src ="files/images/8.png">

Таким образом паук first_spider, пройдёт и соберёт данные со всех страниц.

## Использование атрибутов паука

Вы можете предоставить аргументы командной строки для ваших пауков с помощью -a при их запуске:

In [ ]:
scrapy crawl quotes -o quotes-humor.json -a tag=humor

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "first_spider"

    def start_requests(self):
        url = 'http://quotes.toscrape.com/'
        tag = getattr(self, 'tag', None)
        if tag is not None:
            url = url + 'tag/' + tag
        yield scrapy.Request(url, self.parse)

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('small.author::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

        next_page = response.css('li.next a::attr(href)').extract_first()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

<img src ="files/images/9.png">